In [ ]:
import numpy as np
import pandas as pd
import json
from scipy import optimize
import copy

In [2]:
'''
get_ra(R,P,k)                   # ra= (k[0]*R[0]*R[1])-(k[1]*P[0]*P[1]) # Get the concentration of the R reactant in the reaction k[R]-k[P]
get_RP(eq_name,dict)            # R,P =[[],[]] # CO+H->COH Get the concentration of the R reactant in a reaction. dict{"CO":1} is a dictionary of the concentrations of all molecules
find_k(df_Eak,eq_1)             # Given a set of equations, find the corresponding k
get_ads(eq_name)                # ads_name,moles_name #Identify adsorbates and small molecules from the reaction equation
initial_ads(eqs)                # dict #Given a set of equations, initialize the adsorbate of the reaction to 0
ad_bound(y,dict_bound)          # dict #Add boundary conditions (dictionary)
get_eq_ra(eq,y,df_Eak)          # df #Given a reaction eq, concentration dictionary y, get a table like this columns=["eq","R","P","k","ra"]
R_net                           # fsolve's reaction network
get_df_one(df_all,slab,eta,pH)  # df #Get a reaction network for a kinetic model
MK(eqs,df,dict_bound) ads,R_MK  # Get distribution and reaction rate
'''

def get_RP(eq_name,dict):
    R_name,P_name = eq_name.split("-")
    R_name = R_name.split("+")
    P_name = P_name.split("+")
    R = {}
    for R_n in  R_name:
        R[R_n] = dict[R_n]
    P = {}
    for P_n in  P_name:
        P[P_n] = dict[P_n]

    return R,P

def find_k(df_Eak,eq_1):
    eq_name = eq_1.split("-")
    R_name = sorted(eq_name[0].split("+"))
    P_name = sorted(eq_name[1].split("+"))
    R_ns = df_Eak["R"].values
    P_ns = df_Eak["P"].values
    number = None
    for i in range(len(df_Eak)):
        R_n = sorted(R_ns[i].split(","))
        P_n = sorted(P_ns[i].split(","))
        if R_n == R_name and P_n == P_name:
            number = i
    if number  is None:
        print(f"eq cannot find k in the table,{eq_1}")
    k = df_Eak["k"].values[number]
    k = json.loads(k)
    Ra_name = ""
    if len(R_name) == 1:
        Ra_name = R_name[0]
    else:
        Ra_name = R_name[0] + "+"+ R_name[1]
    Pa_name = ""
    if len(P_name) == 1:
        Pa_name = P_name[0]
    else:
        Pa_name = P_name[0] + "+" + P_name[1]
    K = {}
    K[f"{Ra_name}-{Pa_name}"] = k[0]
    K[f"{Pa_name}-{Ra_name}"] = k[1]
    return K

def get_ra(R,P,k):
    R_values = list(R.values())
    P_values = list(P.values())
    if len(R_values) == 1:
        R["1"] = 1
    if len(P_values) == 1:
        P["1"] = 1
    R_keys = list(R.keys()) 
    R_values = list(R.values())
    P_keys = list(P.keys()) 
    P_values = list(P.values())
    k_values = list(k.values())
    k_keys = list(k.keys())
    ri_value = (k_values[0]*R_values[0]*R_values[1])-(k_values[1]*P_values[0]*P_values[1])
    ri_name = f"({k_keys[0]}.{R_keys[0]}.{R_keys[1]}-{k_keys[1]}.{P_keys[0]}.{P_keys[1]})"
    Ri = {ri_name:ri_value}
    return Ri 

def get_ads(eq_name):
    R_name,P_name = eq_name.split("-")
    R_name = R_name.split("+")
    P_name = P_name.split("+")
    names = list(tuple(R_name)+tuple(P_name))
    ads_name = []
    moles_name = []
    for name in names:
        if len(name)>1 and "*" in name:
            ads_name.append(name)
        else:
            if name != "*":
                moles_name.append(name)
    return ads_name,moles_name

def initial_ads(eqs):
    if len(eqs) == 1:
        print("eqs == 1")
    ads_all = ()
    for i in range(len(eqs)):
        ads_name = get_ads(eqs[i])[0]
        ads_all = ads_all+tuple(ads_name)
    my_list = list(ads_all)
    unique_list = []
    for item in my_list:
        if item not in unique_list:
            unique_list.append(item)
    dict = {}
    for i in range(len(unique_list)):
        dict[unique_list[i]] = 0
    return dict

def ad_bound(y,dict_bound):
    for key, value in dict_bound.items():
        y[key] = value
    return y

def get_RP_from_ads_moles(eq,ads_value,dict_ads,dict_bound):
    eq_name = eq.split("-")
    R_name = sorted(eq_name[0].split("+"))
    P_name = sorted(eq_name[1].split("+"))
    star_ = 1 - sum(ads_value)
    ads_key = list(dict_ads.keys())
    mols_key = list(dict_bound.keys())
    R = {}
    for R_n in R_name:
        if R_n == "*":
            R["*"] = star_
        if R_n in ads_key:
            n = ads_key.index(R_n)
            R[R_n] = (ads_value[n])
        if R_n in mols_key:
            R[R_n] = (dict_bound[R_n])
    P = {}
    for P_n in P_name:
        if P_n == "*":
            P["*"] = star_
        if P_n in ads_key:
            n = ads_key.index(P_n)
            P[P_n] = (ads_value[n])
        if P_n in mols_key:
            P[P_n] = (dict_bound[P_n])
    
    return R,P

def R_net_fslove(ads_value,dict_ads,eqs,dict_bound,df_Eak,A): 
    r_ads = []
    ads_names = list(dict_ads.keys())
    star_ = 1 - sum(ads_value) 
    dict_bound["*"] = star_
    for ads_name in  ads_names:
        Ri = {}
        for eq in eqs:     
            eq_name = eq.split("-")
            R_name = sorted(eq_name[0].split("+"))
            P_name = sorted(eq_name[1].split("+"))
            if ads_name in R_name:
                R,P = get_RP_from_ads_moles(eq,ads_value,dict_ads,dict_bound)
                k = find_k(df_Eak,eq)
                k_keys = list(k.keys())
                k_values = list(k.values())
                k_values = [x * A for x in k_values]
                k = dict(zip(k_keys, k_values))
                ri = get_ra(R,P,k)
                ri_values = list(ri.values())[0]
                ri_keys = "-"+list(ri.keys())[0]
                Ri[ri_keys] = -ri_values
                
            if ads_name in P_name:
                R,P = get_RP_from_ads_moles(eq,ads_value,dict_ads,dict_bound)
                k = find_k(df_Eak,eq)
                k_keys = list(k.keys())
                k_values = list(k.values())
                k_values = [x * A for x in k_values]
                k = dict(zip(k_keys, k_values))
                ri = get_ra(R,P,k)
                ri_values = list(ri.values())[0]
                ri_keys = list(ri.keys())[0]
                Ri[ri_keys] = ri_values
        Ri_values = list(Ri.values())
        Rads_value = sum(Ri_values)
        r_ads.append(Rads_value)
    return r_ads  

def R_net(ads_value,dict_ads,eqs,dict_bound,df_Eak,A): 
    R_net = []
    dict_ads_moles = ad_bound(copy.deepcopy(dict_ads),dict_bound)
    ads_names = list(dict_ads_moles.keys())
    star_ = 1 - sum(ads_value) 
    dict_bound["*"] = star_
    for ads_name in  ads_names:
        Ri = {}
        for eq in eqs:     
            eq_name = eq.split("-")
            R_name = sorted(eq_name[0].split("+"))
            P_name = sorted(eq_name[1].split("+"))
            if ads_name in R_name:
                R,P = get_RP_from_ads_moles(eq,ads_value,dict_ads,dict_bound)
                k = find_k(df_Eak,eq)
                k_keys = list(k.keys())
                k_values = list(k.values())
                k_values = [np.float(x * A) for x in k_values]
                k = dict(zip(k_keys, k_values))
                ri = get_ra(R,P,k)
                ri_values = list(ri.values())[0]
                ri_keys = "-"+list(ri.keys())[0]
                Ri[ri_keys] = -ri_values
                
            if ads_name in P_name:
                R,P = get_RP_from_ads_moles(eq,ads_value,dict_ads,dict_bound)
                k = find_k(df_Eak,eq)
                k_keys = list(k.keys())
                k_values = list(k.values())
                k_values = [x * A for x in k_values]
                k = dict(zip(k_keys, k_values))
                ri = get_ra(R,P,k)
                ri_values = list(ri.values())[0]
                ri_keys = list(ri.keys())[0]
                Ri[ri_keys] = ri_values           
        try:        
            Ri_keys = list(Ri.keys())
            Ri_values = list(Ri.values())
            Rads_value = sum(Ri_values)
            Rads_key = Ri_keys[0]
            if len(Ri_keys) != 1:
                for Ri_key in Ri_keys[1:]:
                    Rads_key = f"{Rads_key} + {Ri_key}"
            a = {Rads_key:Rads_value}
            b = [ads_name,a]
            R_net.append(b)
        except : 
            print(f"{ads_name} is not in eqs")
        

    return R_net 

def get_df_one(df_all,slab,eta,pH):
    df_one = df_all[df_all["slab"] == slab]
    df_one = df_one[df_one["eta"] == eta]
    df_one = df_one[df_one["pH"] == pH]
    return df_one

def MK(dict_ads,eqs,df,dict_bound):
    ads_value = list(dict_ads.values())
    ads_value = [0]*len(ads_value)
    A = 1
    ads_dis = optimize.fsolve(R_net_fslove, ads_value, args=(dict_ads,eqs,dict_bound,df,A))
    ads_keys = list(dict_ads.keys())
    dict_ads_2 = dict(zip(ads_keys, ads_dis))
    A = 6212437991620.12
    ads_dis = optimize.fsolve(R_net_fslove, ads_dis, args=(dict_ads_2,eqs,dict_bound,df,A))
    ads_keys = list(dict_ads.keys())
    dict_ads_MK = dict(zip(ads_keys, ads_dis))
    Ras = R_net(ads_dis,dict_ads_MK,eqs,dict_bound,df,A)

    return dict_ads_MK,ads_dis, Ras

def write_net(ads_value,dict_ads,eqs,dict_bound,df_Eak,A): 
    star_ = 1 - sum(ads_value) 
    dict_bound["*"] = star_
    k_nums = []
    k_re_names =[]
    k_ox_names =[]
    for i,eq in enumerate(eqs):
        k_num = {}
        k = find_k(df_Eak,eq)
        k_values = list(k.values())
        k_values = [x * A for x in k_values]
        k_num[f"kre_{i}"] = k_values[0]
        k_num[f"kox_{i}"] = k_values[1]
        k_nums.append(k_num)
        k_re_names.append(f"kre_{i}")
        k_ox_names.append(f"kox_{i}")
         
    r_ads = []
    ######################################################################################
    str_k_1 = f"\n\tk_re_values = []\n\tk_ox_values = []\n\tfor k_num in k_nums:\n\t\tk_num_values = list(k_num.values())\n\t\tk_re_values.append(k_num_values[0])\n\t\tk_ox_values.append(k_num_values[1])"
    str_t_re = str(k_re_names)[1:-1].replace("'","")
    str_k_2 = f"\n\t{str_t_re} = k_re_values" 
    str_t_ox = str(k_ox_names)[1:-1].replace("'","")
    str_k_3 = f"\n\t{str_t_ox} = k_ox_values" 
    py_k = str_k_1 + str_k_2 + str_k_3

    ads_names = list(dict_ads.keys())
    py_ads_n = str(ads_names)[1:-1].replace("*","a_").replace("'","") +f" = ads_value"
    bound_keys = list(dict_bound.keys())

    s_bound = str(bound_keys)[1:-1].replace("*","a_").replace("'","").replace('(g)', '_g').replace('(l)', '_l')
    print(dict_bound)
    py_bound = f"{s_bound} = list(dict_bound.values())"
    a_ = f"1"
    for ads_name in ads_names:
        ads_name = ads_name.replace("*","a_")
        a_ = f"{a_} - {ads_name}" 
    py_star = f"a_ = {a_}" 

    py_func = "def reaction_net(ads_value,k_nums,dict_bound):" + py_k + "\n\t"+py_ads_n+"\n\t"+py_bound+"\n\t"+py_star

    ######################################################################################

    
    dc_dt_names = []
    for ads_name in ads_names:
        Ri = {}
        for i,eq in enumerate(eqs):     
            eq_name = eq.split("-")
            R_name = sorted(eq_name[0].split("+"))
            P_name = sorted(eq_name[1].split("+"))
            
            if ads_name in R_name:
                R,P = get_RP_from_ads_moles(eq,ads_value,dict_ads,dict_bound)
                k = k_nums[i]
                ri = get_ra(R,P,k)
                ri_values = list(ri.values())[0]
                ri_keys = "-"+list(ri.keys())[0]
                Ri[ri_keys] = -ri_values

                
            if ads_name in P_name:
                R,P = get_RP_from_ads_moles(eq,ads_value,dict_ads,dict_bound)
                k = k_nums[i]
                ri = get_ra(R,P,k)
                ri_values = list(ri.values())[0]
                ri_keys = "+"+list(ri.keys())[0]
                Ri[ri_keys] = ri_values

        Ri_values = list(Ri.values())
        Rads_value = sum(Ri_values)
        r_ads.append(Rads_value)

        Ri_keys = list(Ri.keys())
        Ri_eq = ""
        for ri_key in Ri_keys: 
            Ri_eq= Ri_eq + ri_key
        Ri_eq = Ri_eq.replace('*', 'a_').replace('(g)', '_g').replace('(l)', '_l')
        Ri_eq = Ri_eq.replace('.', '*')
        dc_dt_name = f"d{ads_name[1:]}_dt"
        dc_dt_names.append(dc_dt_name)
        dc_dt = f"\n\t{dc_dt_name} = {Ri_eq}"
        py_func = py_func +dc_dt
    dc_dt_names = str(dc_dt_names).replace("'","")
    py_func =py_func + f"\n\treturn {dc_dt_names}"
    return py_func,k_nums

def get_eq_ra(eqs,dict_ads,dict_bound,df_Eak):
    A = 6212437991620.12
    datas = []
    for eq in eqs:
        eq_name = eq.split("-")
        R_name = sorted(eq_name[0].split("+"))
        P_name = sorted(eq_name[1].split("+"))
        dict_ads_moles = ad_bound(copy.deepcopy(dict_ads),dict_bound)

        ads_value = list(dict_ads.values())
        star_ = 1 - sum(ads_value)

        dict_ads_moles["*"] = star_
        R = {}
        P = {}
        for r_n in R_name:
            R[r_n]= dict_ads_moles[r_n]
        for p_n in P_name:
            P[p_n]= dict_ads_moles[p_n]
        k = find_k(df_Eak,eq)
        ra = get_ra(R,P,k)
        for key,value in ra.items():
            ra[key] = value*A
        row_data = [eq,R,P,k,ra]
        datas.append(row_data)
    #df = pd.DataFrame(datas, columns=["eq","R","P","k","ra"])
    return datas

def cal_coverge(df_1,dict_bound,ads):
    eqs = df_1["Reaction"].tolist()
    df_1["R"] = df_1["Reaction"].apply(lambda x: str(x.split("-")[0].split("+")).replace('[', '').replace(']', '').replace("'", '').replace(" ", '') if isinstance(x, str) else None)
    df_1["P"] = df_1["Reaction"].apply(lambda x: str(x.split("-")[1].split("+")).replace('[', '').replace(']', '').replace("'", '').replace(" ", '') if isinstance(x, str) else None)
    df_1["k"] = df_1['k'].apply(lambda x: f"[{x}]" if isinstance(x, str) else None)
    dict_ads = initial_ads(eqs)
    coverage,list_ads,list_dict_R =MK(dict_ads,eqs,df_1,dict_bound)
    r_a = {}
    for lis in list_dict_R:
        if lis[0] in ads:
            r_a[lis[0]] =list(lis[1].values())[0]
    rate = {x:r_a[x] for x in ads}

    return coverage,rate

In [3]:
df_1 = pd.read_excel("./MK.csv")
dict_bound ={"CO(g)":1,"H":1,"H2O(l)":1,"CH2O(g)":0,"CH4(g)":0,"CH3OH(l)":0, "*":1}
mol = ["CO(g)","CH2O(g)","CH3OH(l)","CH4(g)"]

coverage,rate = cal_coverge(df_1,dict_bound,mol)
coverage,rate

C:\Users\Administrator\AppData\Local\Temp\ipykernel_44896\1414459085.py:197: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  k_values = [np.float(x * A) for x in k_values]


({'*CO': 0.99999940950084,
  '*CHO': 2.3788206369062645e-08,
  '*COH': 9.954875181021881e-08,
  '*CH2O': 1.1957399836290167e-09,
  '*CHOH': 1.4706302390700794e-09,
  '*C': 1.5866463483927363e-15,
  '*CH3O': 7.808472280956078e-10,
  '*CH2OH': 4.961567283814436e-12,
  '*CH': 2.439157792368483e-11,
  '*O': 5.46633166412774e-17,
  '*CH3OH': 1.490154729806936e-11,
  '*CH2': 1.1583625472176548e-10,
  '*OH': 1.3520317618570465e-13,
  '*CH3': 4.634329489303823e-07,
  '*H2O': 1.327461509499711e-16,
  '*CH4': 1.1248104098883856e-10},
 {'CO(g)': -57.90226565297512,
  'CH2O(g)': 0.4758131080692412,
  'CH3OH(l)': 0.47069153918111206,
  'CH4(g)': 56.955428820748324})